# ViT 2: Electric Transformaroo
We learned a few lessons from the previous experiment. First, and primarily, a larger model doesn't always mean better. We used the large imageNet transformer model, but that didn't actually lead to good predictions. Also, I think I screwed up a bunch of stuff, and the large model is taking way too long to train for me to iterate at all. We are going to switch to a smaller model. We are going to follow the ViTMAE method, use masked auto encodings for pretraining in a self-supervised manner on the entire training set, then fine tune on the prediction task. Guide found here: https://github.com/huggingface/transformers/blob/main/examples/pytorch/image-pretraining/run_mae.py 

This notebook has the fine-tuning process, fine tuning on Binary CE Loss.

In [2]:
# imports
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import torch
print(torch.cuda.is_available())

from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTMAEForPreTraining, ViTMAEConfig
from transformers import TrainingArguments, Trainer
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop
from torchvision.transforms.functional import InterpolationMode

from sklearn.model_selection import train_test_split

True


In [3]:
from dataloader import *
from utils import *
from trainer import *

In [4]:
# CONSTANTS
FEATURE_EXTRACTOR_NAME = 'facebook/vit-mae-base'
VIT_MODEL_NAME = 'vit-mae-chexpert/'
TRAIN_SPLIT = 0.8
BATCH_SIZE = 48
LEARNING_RATE = 1.5e-2
LR_SCHEDULER_TYPE = "cosine"
WEIGHT_DECAY = 1e-5
WARMUP_RATIO = 0.05
LOGGING_STRATEGY = "steps"
LOGGING_STEPS = 10
FP16 = True
EPOCHS = 10
EVALUATION_STRATEGY = "steps"
EVAL_STEPS = 500
OUTPUT_DIR = './vit-mae-chexpert-auc-fine-tuned-1'
REMOVE_UNUSED_COLUMNS = False
GRAD_ACCUM_STEPS = 5
DATALOADER_NUM_WORKERS = 4
MARGIN = 1.0
GAMMA = 500

In [5]:
feature_extractor = ViTFeatureExtractor.from_pretrained(FEATURE_EXTRACTOR_NAME, image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

In [6]:
# set up our transforms
transforms = [
    RandomResizedCrop(feature_extractor.size, scale=(0.2, 1.0), interpolation=InterpolationMode.BICUBIC),
    RandomHorizontalFlip(),
]

In [7]:
np.random.seed(42)
train_df = pd.read_csv("ChexPert/train.csv")
train_df, eval_df = train_test_split(train_df, train_size=TRAIN_SPLIT)

train_dataset = ChexpertViTDataset("ChexPert/data", train_df, feature_extractor, transforms=transforms, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)
eval_dataset = ChexpertViTDataset("ChexPert/data", eval_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)

In [8]:
imratio=[0.1497333982353369,
 0.12095831165270714,
 0.0663119436471569,
 0.2344081328924473,
 0.3860606162333339]

In [9]:
model = ViTForImageClassification.from_pretrained(
    VIT_MODEL_NAME,
    num_labels=len(train_dataset.labels)
).to("cuda")

You are using a model of type vit_mae to instantiate a model of type vit. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vit-mae-chexpert/ were not used when initializing ViTForImageClassification: ['decoder.decoder_layers.2.intermediate.dense.weight', 'decoder.decoder_layers.3.attention.attention.key.weight', 'decoder.decoder_layers.2.attention.attention.value.bias', 'decoder.decoder_layers.7.layernorm_before.weight', 'decoder.decoder_layers.2.output.dense.bias', 'decoder.decoder_embed.weight', 'decoder.decoder_layers.1.intermediate.dense.weight', 'decoder.decoder_layers.6.intermediate.dense.bias', 'decoder.decoder_layers.7.output.dense.bias', 'decoder.decoder_layers.6.layernorm_before.bias', 'decoder.decoder_layers.0.intermediate.dense.weight', 'decoder.decoder_layers.6.layernorm_after.weight', 'decoder.decoder_layers.5.attention.attention.value.bias', 'decoder.decoder_layers.3.attention.attention.query.bias', 'dec

In [10]:
# set up training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy=EVALUATION_STRATEGY,
    num_train_epochs=EPOCHS,
    fp16=FP16,
    eval_steps = EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    remove_unused_columns=REMOVE_UNUSED_COLUMNS,
    report_to="tensorboard",
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    logging_strategy=LOGGING_STRATEGY,
    logging_steps=LOGGING_STEPS,
    dataloader_num_workers=DATALOADER_NUM_WORKERS
)

In [11]:
trainer = AUCTrainer(
    imratio=imratio,
    margin=MARGIN,
    gamma=GAMMA,
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

Using amp half precision backend


In [12]:
trainer.train()

***** Running training *****
  Num examples = 152878
  Num Epochs = 10
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 240
  Gradient Accumulation steps = 5
  Total optimization steps = 6370


Step,Training Loss,Validation Loss,Accuracy,F1,Auc Atelectasis,Auc Cardiomegaly,Auc Consolidation,Auc Edema,Auc Pleural effusion,Average Auc
500,0.608200,0.607641,0.241238,0.061366,0.576434,0.581649,0.586836,0.718703,0.665067,0.625738
1000,0.485300,0.482056,0.208577,0.500868,0.585232,0.581121,0.615586,0.738934,0.738449,0.651864
1500,0.407800,0.438065,0.158641,0.525277,0.601730,0.600357,0.625187,0.735529,0.769881,0.666537
2000,0.422700,0.411660,0.126006,0.531817,0.608557,0.623940,0.629073,0.745386,0.789054,0.679202
2500,0.372700,0.384966,0.101025,0.530692,0.613929,0.660903,0.632036,0.759810,0.802452,0.693826
3000,0.346600,0.372426,0.083384,0.532476,0.616916,0.704875,0.631378,0.769393,0.804579,0.705428
3500,0.352700,0.369217,0.069281,0.523450,0.620489,0.718184,0.631955,0.770395,0.809541,0.710113
4000,0.352300,0.357970,0.087499,0.537406,0.615384,0.733095,0.631420,0.780168,0.809193,0.713852
4500,0.345700,0.353422,0.086949,0.535176,0.617588,0.741457,0.631986,0.781762,0.813627,0.717284
5000,0.314300,0.351640,0.084170,0.539178,0.617582,0.746607,0.631828,0.784467,0.813474,0.718791


***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48
Saving model checkpoint to ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-500
Configuration saved in ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-500/config.json
Model weights saved in ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48
Saving model checkpoint to ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-1000
Configuration saved in ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-1000/config.json
Model weights saved in ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48
Saving model checkpoint to ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-1500
Configuration saved in ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-1500/config.json
Model weights saved in ./vit-mae-chexpert-auc-fine-tuned-1/checkpoint-1500/pytorch_model.bin
***** Runn

TrainOutput(global_step=6370, training_loss=0.40268597415702495, metrics={'train_runtime': 23295.034, 'train_samples_per_second': 65.627, 'train_steps_per_second': 0.273, 'total_flos': 1.184713892621148e+20, 'train_loss': 0.40268597415702495, 'epoch': 10.0})

In [13]:
trainer.save_model()

Saving model checkpoint to ./vit-mae-chexpert-auc-fine-tuned-1
Configuration saved in ./vit-mae-chexpert-auc-fine-tuned-1/config.json
Model weights saved in ./vit-mae-chexpert-auc-fine-tuned-1/pytorch_model.bin


In [14]:
validation_df = pd.read_csv("ChexPert/valid.csv")
valid_dataset = ChexpertViTDataset("ChexPert/data", validation_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)

In [15]:
trainer.evaluate(eval_dataset=valid_dataset)

***** Running Evaluation *****
  Num examples = 202
  Batch size = 48


{'eval_loss': -0.1631758064031601,
 'eval_accuracy': 0.15346534653465346,
 'eval_f1': 0.585427135678392,
 'eval_AUC_Atelectasis': 0.8475590551181102,
 'eval_AUC_Cardiomegaly': 0.7463235294117646,
 'eval_AUC_Consolidation': 0.9041360294117646,
 'eval_AUC_Edema': 0.8631696428571429,
 'eval_AUC_Pleural Effusion': 0.8650928442028987,
 'eval_average_auc': 0.8452562202003362,
 'eval_runtime': 1.9885,
 'eval_samples_per_second': 101.583,
 'eval_steps_per_second': 2.514,
 'epoch': 10.0}